In [148]:
from distutils.log import info
import datetime
import pandas as pd
import numpy as np
from  SQL_e import sql_datenLadenLabel,sql_datenLadenOderItems,sql_datenLadenStammdaten,sql_datenLadenOder,createnewTable

def stammdatenBearbeiten():
    dfStammdaten = sql_datenLadenStammdaten()
    dfStammdaten = dfStammdaten[dfStammdaten['UnitOfMeasure'].isin(['CS','D97','OUT'])]   
    dfStammdaten['MaterialNumber'] = dfStammdaten['MaterialNumber'].str.replace('0000000000', '')
    
    def f_CS(row):
        try:
            if row.UnitOfMeasure == 'CS':          
                return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
        except:
            return np.nan
    def f_PAL(row):
        try:
            if row.UnitOfMeasure == 'D97':
                return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
        except:
            return np.nan
    def f_OUT(row):
        try:
            if row.UnitOfMeasure == 'OUT':
                return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
        except:
            return np.nan
    dfStammdaten['OUT'] = dfStammdaten.apply(f_OUT,axis=1)
    dfStammdaten['CS'] = dfStammdaten.apply(f_CS,axis=1)
    dfStammdaten['PAL'] = dfStammdaten.apply(f_PAL,axis=1)

    return dfStammdaten



In [149]:
dfOrderItems = sql_datenLadenOderItems()

In [150]:

dfOrder = sql_datenLadenOder()

In [151]:
dfStammdaten = stammdatenBearbeiten()

In [152]:
###TODO Die zusammenfassung der Items in der Datenbank verfälschen die Anzahl Picks PAL und CS
### da die Positionen einzeln gepickt werden sind sie nicht als BS3 Pick navigiert und damit kein Pal Pick 

dfOrderItems['MaterialNumber'] = dfOrderItems['MaterialNumber'].astype(str)
dfOrderItems['MaterialNumber'] = dfOrderItems['MaterialNumber'].str.replace('0000000000', '')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'CS'][['MaterialNumber','CS']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'D97'][['MaterialNumber','PAL']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'OUT'][['MaterialNumber','OUT']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')

dfOrderItems['O'] = dfOrderItems['Outers'] * dfOrderItems['OUT']
dfOrderItems['PicksPAL'] = dfOrderItems.O / dfOrderItems.PAL
dfOrderItems['PicksCS'] = dfOrderItems.CorrespondingCartons 
dfOrderItems['Picks OUT'] = dfOrderItems.CorrespondingOuters
#Bereinige Berechnungen der Picks 
dfOrderItems['PicksPAL'] = dfOrderItems['PicksPAL'].fillna(0) 
dfOrderItems['O'] = dfOrderItems.O.fillna(0) 
dfOrderItems['Picks OUT'] = dfOrderItems['Picks OUT'].fillna(0) 

# Pal kleiner als 1 bereinigen
def f_PAL(row):
        if row.PicksPAL < 1:
            return 0
        else:
            return row.PicksPAL

dfOrderItems['Picks PAL'] = dfOrderItems.apply(f_PAL,axis=1)

def f_OUT(row):
        if row.PicksPAL >= 1:
            return row.CorrespondingCartons  - ((row.PicksPAL * row.PAL) / row.CS)
        else:
            return row.CorrespondingCartons

dfOrderItems['Picks CS'] = dfOrderItems.apply(f_OUT,axis=1)

dfOrderItems['Picks Gesamt'] = dfOrderItems['Picks PAL'] + dfOrderItems['Picks CS'] + dfOrderItems['Picks OUT']
df = pd.merge(dfOrder, dfOrderItems, left_on='SapOrderNumber', right_on='SapOrderNumber', how='left')

# sap order 3202209324
# h = dfOrderItems[dfOrderItems['SapOrderNumber'] == '3202209324']
# h

In [153]:
vdef = dfOrderItems[dfOrderItems['SapOrderNumber'] == '3202200999']
vdef 

,SapOrderNumber,ItemNumber,KVP,Outers,CorrespondingCartons,CorrespondingOuters,CorrespondingPackages,PackagesPerOuter,InternationalArticleNumberForOuter,MaterialNumber,...,CS,PAL,OUT,O,PicksPAL,PicksCS,Picks OUT,Picks PAL,Picks CS,Picks Gesamt
63748,3202200999,000010,7.50,200,20,0,0,5,4031300063279,50063261,...,10.00,200.00,1.000,200.00,1.000000,20,0,1.0,0.0,1.0
63749,3202200999,000020,7.50,200,20,0,0,5,4031300063286,50063231,...,10.00,200.00,1.000,200.00,1.000000,20,0,1.0,0.0,1.0
63750,3202200999,000030,76.00,1050,42,0,0,10,4031300267059,10139133,...,5.00,210.00,0.200,210.00,1.000000,42,0,1.0,0.0,1.0
63751,3202200999,000040,64.00,825,33,0,0,8,4031300268384,10158908,...,4.80,201.60,0.192,158.40,0.785714,33,0,0.0,33.0,33.0
63752,3202200999,000050,72.00,1025,41,0,0,10,4031300268353,10139454,...,5.00,210.00,0.200,205.00,0.976190,41,0,0.0,41.0,41.0
63753,3202200999,000060,64.00,1050,42,0,0,8,4031300268445,10158927,...,4.80,201.60,0.192,201.60,1.000000,42,0,1.0,0.0,1.0
63754,3202200999,000070,80.00,1680,84,0,0,8,4031300278161,10158284,...,4.96,208.32,0.248,416.64,2.000000,84,0,2.0,0.0,2.0
63755,3202200999,000080,39.95,70,7,0,0,1,4031300278703,10160191,...,1.95,68.25,0.195,13.65,0.200000,7,0,0.0,7.0,7.0


In [159]:
sumPicks = vdef['Picks Gesamt'].sum()
sumPicks

87.0

In [160]:
# def f_OUT(row):
#     try:
#         if row.PicksPAL >= 1:
#             return ((row.PicksPAL * row.PAL) - row.O) 
#     except:
#         return np.nan
# vdef['PickPal'] = vdef.apply(f_OUT,axis=1)
vdef

,SapOrderNumber,ItemNumber,KVP,Outers,CorrespondingCartons,CorrespondingOuters,CorrespondingPackages,PackagesPerOuter,InternationalArticleNumberForOuter,MaterialNumber,...,CS,PAL,OUT,O,PicksPAL,PicksCS,Picks OUT,Picks PAL,Picks CS,Picks Gesamt
63748,3202200999,000010,7.50,200,20,0,0,5,4031300063279,50063261,...,10.00,200.00,1.000,200.00,1.000000,20,0,1.0,0.0,1.0
63749,3202200999,000020,7.50,200,20,0,0,5,4031300063286,50063231,...,10.00,200.00,1.000,200.00,1.000000,20,0,1.0,0.0,1.0
63750,3202200999,000030,76.00,1050,42,0,0,10,4031300267059,10139133,...,5.00,210.00,0.200,210.00,1.000000,42,0,1.0,0.0,1.0
63751,3202200999,000040,64.00,825,33,0,0,8,4031300268384,10158908,...,4.80,201.60,0.192,158.40,0.785714,33,0,0.0,33.0,33.0
63752,3202200999,000050,72.00,1025,41,0,0,10,4031300268353,10139454,...,5.00,210.00,0.200,205.00,0.976190,41,0,0.0,41.0,41.0
63753,3202200999,000060,64.00,1050,42,0,0,8,4031300268445,10158927,...,4.80,201.60,0.192,201.60,1.000000,42,0,1.0,0.0,1.0
63754,3202200999,000070,80.00,1680,84,0,0,8,4031300278161,10158284,...,4.96,208.32,0.248,416.64,2.000000,84,0,2.0,0.0,2.0
63755,3202200999,000080,39.95,70,7,0,0,1,4031300278703,10160191,...,1.95,68.25,0.195,13.65,0.200000,7,0,0.0,7.0,7.0


In [161]:
# dfOrderItems['O'] = dfOrderItems['Outers'] * dfOrderItems['OUT']
# dfOrderItems['Picks PAL'] = dfOrderItems.O / dfOrderItems.PAL
# dfOrderItems['Picks CS'] = dfOrderItems.CorrespondingCartons 